In [213]:
import spacy
import codecs
from spacy import displacy
import os
import matplotlib as plt
from collections import Counter

мне нужно так обработать текст, чтобы по нему потом можно было, например, выделять ошибки

In [125]:
nlp = spacy.load('ru_core_news_sm')
# print(token.text, token.pos_, token.dep_)


In [126]:
nlp.add_pipe('sentencizer')
nlp.disable_pipe("lemmatizer")

In [127]:
def opener(adress):
    file = codecs.open(adress, "r", "utf-8")
    text = file.read()
    file.close()
    return text    

In [128]:
def cleaner(text: str):
    new = text.replace("\n", ".").replace("\t", "").replace("\r", "").replace("\xa0", " ").replace("I", "")
    return new


In [129]:
d = {}
for filename in os.scandir("6_texts"):
        d["{}".format(filename.name)] = nlp(cleaner(opener(filename.path)))

Программа будет выводить <br>
количество простых, ССП, СПП предложений <br>
количество восклицательных, вопросительных, утвердительных <br>
типы словосочетний - согласование, управление, примыкание (?)

In [130]:
#если у других глаголов не nsubj - оно простое
#если в предложении у глаголов кроме root'а есть nsubj, то это предложение сложное
#если между частями сложного предложения есть союзы соединительные, противительные и разделительные, то это ССП
#если есть союзы СППшные, тогда это СПП
#если нет союзов, но есть две основы, пойдёт в общую кучу БСП

#1 - найти рут
#1.1 - нвйти осталдьные глаголы
# 2 - итерация по тем, которые зависят от глагола
# 3 - если есть nsubj, тогда в копилку сложных
# 3.1 - если нет, в копилку простых
# плюс предлоги СПП

In [202]:
conj = "хотя, несмотря на то что, так как, в связи с тем что, потому что, ибо, чтобы, для того чтобы, \
дабы, когда, в то время как, пока, до тех пор пока, едва, лишь, \
так что, как будто, чем, если, кабы, коли, чтобы"

conj_SPP = conj.split(", ")

In [ ]:
def compound_finder(text):
    text = text.sents
    compound = [] #ССП
    complex = [] #СПП
    simple = []
    for sentence in text:
        n = 0
        for token in sentence:
            if (n!=1) and token.text in conj_SPP:
                complex.append(sentence)
                n+=1
            elif (token.dep_ != "ROOT") and (token.pos_ == "VERB"):
                for child in token.children:
                    if (n!=1) and (child.dep_ == "nsubj"):
                        compound.append(sentence)
                        n+=1        
            elif (n!=1) and token.dep_ == "parataxis":
                compound.append(sentence)
                n+=1
        if sentence not in (compound or complex):
            simple.append(sentence)
        
    return compound, complex, simple

In [226]:
ccs = [] #compound complex simple
for key in d.keys():
    a = len(compound_finder(d[key])[0])
    b = len(compound_finder(d[key])[1])
    c = len(compound_finder(d[key])[2])
    ccs.append([a,b,c])

ccs

[[105, 15, 175],
 [447, 39, 624],
 [229, 16, 249],
 [117, 9, 85],
 [1841, 234, 2014],
 [38, 7, 62]]

In [221]:
f = compound_finder(d["1_anton_yabl_tok_1895.txt"])[0]
len(f)

105

In [178]:



y = [len(compound_finder(d["1_anton_yabl_tok_1895.txt"])[0]), len(compound_finder(d["1_anton_yabl_tok_1895.txt"])[1]),\
                                                                  len(simple(d["1_anton_yabl_tok_1895.txt"]))]




[105, 15, 176]

In [133]:
# doc = d["1_anton_yabl_tok_1895.txt"]
# sentences = [sent.text.strip() for sent in doc.sents] #.text берёт текст от объекта, а стрип удаляет началньые и конечные пробелы
#чтобы он нашёл руты каждого предложения в каждом тексте 

for key in d.keys():
    doc = d[key].sents.text.strip()


AttributeError: 'generator' object has no attribute 'text'

In [144]:
sent = "Покупает, конечно, один, ибо и покупки-то всего на копейку или на яйцо, но покупателей много, торговля идет бойко, и чахоточный мещанин в длинном сюртуке и рыжих сапогах -- весел."
sent = nlp(sent)

displacy.render(sent, style = "dep")